# Testing Deployed Kidney Stone Prediction Pipeline


#### TF Serving URL (Model Prediction)

- http://103.190.215.112:8501/v1/models/kidney-stone-prediction/metadata

#### Prometheus URL (Model Monitoring)

- http://103.190.215.112:9090

#### Grafana URL (Model Monitoring Visualization)

- http://103.190.215.112:3000


In [2]:
import tensorflow as tf
import pandas as pd
import requests
import base64
import json
import os
from pprint import PrettyPrinter

In [3]:
pp = PrettyPrinter()
BASE_URL = "http://103.190.215.112"

## Checking Model Status


In [8]:
pp.pprint(requests.get(f"{BASE_URL}:8501/v1/models/kidney-stone-prediction").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1714916304'}]}


## Testing from Dataset


### Loading the Dataset


In [9]:
DATASET_PATH = os.path.abspath("data/kidney-stone-urine-prediction.csv")

df = pd.read_csv(DATASET_PATH)
df.head()

,gravity,ph,osmo,cond,urea,calc,target
0,1.021,4.91,725,14.0,443,2.45,0
1,1.017,5.74,577,20.0,296,4.49,0
2,1.008,7.20,321,14.9,101,2.36,0
3,1.011,5.51,408,12.6,224,2.15,0
4,1.005,6.52,187,7.5,91,1.16,0


### Creating Samples to Test


In [201]:
samples = df.sample(10, ignore_index=True)
sample_labels = samples["target"]
sample_data = samples.drop("target", axis=1)
sample_data = sample_data.to_dict("records")

In [202]:
sample_data

[{'gravity': 1.023,
  'ph': 5.85,
  'osmo': 970,
  'cond': 38.0,
  'urea': 362,
  'calc': 4.54},
 {'gravity': 1.02,
  'ph': 5.68,
  'osmo': 876,
  'cond': 35.8,
  'urea': 308,
  'calc': 4.49},
 {'gravity': 1.017,
  'ph': 5.74,
  'osmo': 577,
  'cond': 20.0,
  'urea': 296,
  'calc': 4.49},
 {'gravity': 1.022,
  'ph': 6.21,
  'osmo': 442,
  'cond': 20.6,
  'urea': 398,
  'calc': 4.18},
 {'gravity': 1.018,
  'ph': 5.14,
  'osmo': 703,
  'cond': 29.0,
  'urea': 272,
  'calc': 6.63},
 {'gravity': 1.02,
  'ph': 5.44,
  'osmo': 781,
  'cond': 29.0,
  'urea': 349,
  'calc': 3.04},
 {'gravity': 1.025,
  'ph': 7.9,
  'osmo': 721,
  'cond': 23.6,
  'urea': 301,
  'calc': 9.04},
 {'gravity': 1.016,
  'ph': 6.81,
  'osmo': 594,
  'cond': 21.4,
  'urea': 255,
  'calc': 12.2},
 {'gravity': 1.024,
  'ph': 5.6,
  'osmo': 866,
  'cond': 29.5,
  'urea': 360,
  'calc': 5.54},
 {'gravity': 1.013,
  'ph': 5.86,
  'osmo': 531,
  'cond': 21.4,
  'urea': 197,
  'calc': 4.74}]

#### Converting Samples into TF Serving Request Format


In [180]:
def base64_examples(data):
    feature_spec = {}

    features = data.keys()
    for feature in features:
        if type(data[feature]) == int:
            feature_spec[feature] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=[data[feature]])
            )
        elif type(data[feature]) == float:
            feature_spec[feature] = tf.train.Feature(
                float_list=tf.train.FloatList(value=[data[feature]])
            )

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    return {"examples": {"b64": base64.b64encode(example).decode()}}

In [203]:
instances = [base64_examples(data) for data in sample_data]
request_data = json.dumps({"signature_name": "serving_default", "instances": instances})

#### Getting Response


In [204]:
response = requests.post(
    f"{BASE_URL}:8501/v1/models/kidney-stone-prediction:predict",
    data=request_data,
).json()

#### Converting Predictions Response as Label


In [205]:
threshold = 0.5
pred_labels = [
    "Positive" if int(prediction[0] >= threshold) else "Negative"
    for prediction in response.get("predictions")
]

#### Creating DataFrame to Visualize Predictions


In [206]:
result_df = pd.DataFrame(sample_data)
result_df["label"] = [
    "Positive" if label == 1 else "Negative" for label in sample_labels
]
result_df["prediction"] = pred_labels
result_df

,gravity,ph,osmo,cond,urea,calc,label,prediction
0,1.023,5.85,970,38.0,362,4.54,Negative,Negative
1,1.020,5.68,876,35.8,308,4.49,Negative,Negative
2,1.017,5.74,577,20.0,296,4.49,Negative,Negative
3,1.022,6.21,442,20.6,398,4.18,Positive,Positive
4,1.018,5.14,703,29.0,272,6.63,Positive,Positive
5,1.020,5.44,781,29.0,349,3.04,Negative,Negative
6,1.025,7.90,721,23.6,301,9.04,Positive,Positive
7,1.016,6.81,594,21.4,255,12.20,Positive,Positive
8,1.024,5.60,866,29.5,360,5.54,Positive,Negative
9,1.013,5.86,531,21.4,197,4.74,Positive,Positive
